# Домашняя работа

Взять boston house-prices datase (sklearn.datasets.load_boston) и сделать тоже самое для задачи регрессии (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество)

In [1]:
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.linear_model import LinearRegression as lr
from sklearn.neighbors import KNeighborsRegressor as kn
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor as dt
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston as boston
from sklearn.linear_model import ElasticNet as en
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

In [2]:
warnings.filterwarnings("ignore")
models = [lr, rf, kn, dt, en]

In [4]:
y = pd.DataFrame(columns=['MEDV'], data = boston()['target'])
X = pd.DataFrame(columns=list(boston()['feature_names']), data = boston()['data'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [108]:
X

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [70]:
# variable params for GridSearchCV, RandomizedSearchCV
params = {'n_estimators':10**np.arange(1,3), 
          'max_depth':10**np.arange(1,3),
          'criterion':['mse', 'mae'],
          'max_features': ['auto', 'sqrt', 'log2'],
          'weights': ['uniform', 'distance'],
          'class_weight':['balanced', 'balanced_subsample'],
          'n_neighbors': range(1,30),
          'l1_ratio':np.linspace(0,1,5),
          'n_jobs':[-1],
          'normalize':[True, False]}

In [102]:
def param4model(model, params_all, *a, **k): 
    return {key:params_all[key] for key in set(model.get_params().keys()) & set(params_all.keys())}

In [81]:
results = pd.DataFrame(columns=['estimator', 'score', 'params'])
for m in models:
    clf = GridSearchCV(m(), (par:= param4model(m(), params)), cv=10, n_jobs=-1).fit(X_train, y_train)
    print(f'{clf.cv_results_}\n\n{(estimator:= m.__name__)} {(score:= clf.score(X_test, y_test))}\n\n\n\n')
    results = results.append({'estimator':estimator, 'score':score, 'params':clf.best_params_}, ignore_index=True)

results = results.sort_values(by='score', ascending=False).reset_index(drop=True)

{'mean_fit_time': array([0.00880277, 0.01079912]), 'std_fit_time': array([0.00299408, 0.0018325 ]), 'mean_score_time': array([0.00759962, 0.00560167]), 'std_score_time': array([0.00280095, 0.00195717]), 'param_n_jobs': masked_array(data=[-1, -1],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_normalize': masked_array(data=[True, False],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_jobs': -1, 'normalize': True}, {'n_jobs': -1, 'normalize': False}], 'split0_test_score': array([0.78779125, 0.78779125]), 'split1_test_score': array([-0.07880766, -0.07880766]), 'split2_test_score': array([0.75127458, 0.75127458]), 'split3_test_score': array([0.53452948, 0.53452948]), 'split4_test_score': array([0.6687292, 0.6687292]), 'split5_test_score': array([0.43046692, 0.43046692]), 'split6_test_score': array([0.74822572, 0.74822572]), 'split7_test_score': array([0.68387251, 0.68387251]), 'split8_test_score

{'mean_fit_time': array([0.00680082, 0.00800021, 0.00840018, 0.00759983, 0.00840018,
       0.00960052, 0.00959945, 0.00879686, 0.00880024, 0.00880036,
       0.0088001 , 0.00840018, 0.00760012, 0.00840027, 0.00800056,
       0.00840001, 0.00839958, 0.01000004, 0.00800238, 0.00919781,
       0.00959945, 0.00920253, 0.00840254, 0.00919991, 0.00840013,
       0.00840023, 0.00880032, 0.0096    , 0.00880017, 0.00959966,
       0.01000016, 0.00800183, 0.00880015, 0.00840023, 0.00840127,
       0.00800135, 0.00920012, 0.00800064, 0.00840037, 0.00840015,
       0.0092001 , 0.00879984, 0.00960009, 0.00919986, 0.00960028,
       0.00880027, 0.00839972, 0.0088001 , 0.00879996, 0.00920012,
       0.00920014, 0.00959954, 0.00920048, 0.00879986, 0.00800014,
       0.00919988, 0.00880005, 0.00839972]), 'std_fit_time': array([1.83172758e-03, 9.17524405e-07, 1.20012794e-03, 1.20011984e-03,
       1.20012771e-03, 1.95962510e-03, 1.95913845e-03, 1.59457931e-03,
       1.60033718e-03, 1.59991987e-03, 1.6

{'mean_fit_time': array([0.00839727, 0.00639997, 0.00720005, 0.0104003 , 0.01000013,
       0.00759997, 0.03919778, 0.01480227, 0.01719794, 0.03796093,
       0.01599956, 0.01699152]), 'std_fit_time': array([1.19768837e-03, 1.96009225e-03, 1.59987219e-03, 1.95947899e-03,
       5.44042177e-03, 1.20056518e-03, 5.60137304e-03, 1.82968149e-03,
       2.56215651e-03, 9.34122598e-03, 1.09776158e-06, 1.56286718e-03]), 'mean_score_time': array([0.0047998 , 0.00400031, 0.00480008, 0.00599988, 0.00599978,
       0.00600009, 0.0052022 , 0.0059978 , 0.00560207, 0.00414944,
       0.00480049, 0.0060195 ]), 'std_score_time': array([0.00159999, 0.00253005, 0.00159997, 0.00268339, 0.00200007,
       0.00199928, 0.00183648, 0.00268125, 0.00196147, 0.00196698,
       0.00160012, 0.00215172]), 'param_criterion': masked_array(data=['mse', 'mse', 'mse', 'mse', 'mse', 'mse', 'mae', 'mae',
                   'mae', 'mae', 'mae', 'mae'],
             mask=[False, False, False, False, False, False, False, Fal

In [104]:
results_random = pd.DataFrame(columns=['estimator', 'score', 'params'])
for m in models:
    clf = RandomizedSearchCV(m(), (par:= param4model(m(), params)), cv=10, n_jobs=-1).fit(X_train, y_train)
    print(f'{clf.cv_results_}\n\n{(estimator:= m.__name__)} {(score:= clf.score(X_test, y_test))}\n\n')
    results_random = results_random.append({'estimator':estimator, 'score':score, 'params':clf.best_params_}, ignore_index=True)

results_random = results_random.sort_values(by='score', ascending=False).reset_index(drop=True)


{'mean_fit_time': array([0.01520054, 0.00959904]), 'std_fit_time': array([0.00239962, 0.00320011]), 'mean_score_time': array([0.0076005 , 0.00599999]), 'std_score_time': array([0.00215433, 0.00200028]), 'param_normalize': masked_array(data=[True, False],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_n_jobs': masked_array(data=[-1, -1],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'normalize': True, 'n_jobs': -1}, {'normalize': False, 'n_jobs': -1}], 'split0_test_score': array([0.78779125, 0.78779125]), 'split1_test_score': array([-0.07880766, -0.07880766]), 'split2_test_score': array([0.75127458, 0.75127458]), 'split3_test_score': array([0.53452948, 0.53452948]), 'split4_test_score': array([0.6687292, 0.6687292]), 'split5_test_score': array([0.43046692, 0.43046692]), 'split6_test_score': array([0.74822572, 0.74822572]), 'split7_test_score': array([0.68387251, 0.68387251]), 'split8_test_score

{'mean_fit_time': array([0.0411998 , 0.01560178, 0.01599963, 0.01719985, 0.01599905,
       0.00839965, 0.00839992, 0.03319948, 0.01239953, 0.00879986]), 'std_fit_time': array([5.38199715e-03, 2.15631218e-03, 2.52971425e-03, 3.12360487e-03,
       5.97950754e-07, 1.19847543e-03, 1.20005646e-03, 1.18731411e-02,
       4.17863447e-03, 2.99373981e-03]), 'mean_score_time': array([0.00600076, 0.0071985 , 0.00560002, 0.00640049, 0.0064008 ,
       0.01200027, 0.00800076, 0.00559995, 0.00639935, 0.0055999 ]), 'std_score_time': array([0.00200005, 0.00239982, 0.0019592 , 0.00319974, 0.00195961,
       0.01290104, 0.00438164, 0.00319953, 0.00265285, 0.00195973]), 'param_max_features': masked_array(data=['auto', 'sqrt', 'log2', 'sqrt', 'log2', 'sqrt', 'log2',
                   'auto', 'auto', 'log2'],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[

In [101]:
#best results for GridSearchCV
results

,estimator,score,params
0,RandomForestRegressor,0.872792,"{'criterion': 'mae', 'max_depth': 100, 'max_fe..."
1,DecisionTreeRegressor,0.750974,"{'criterion': 'mse', 'max_depth': 100, 'max_fe..."
2,LinearRegression,0.730396,"{'n_jobs': -1, 'normalize': True}"
3,ElasticNet,0.671531,"{'l1_ratio': 0.0, 'normalize': False}"
4,KNeighborsRegressor,0.496019,"{'n_jobs': -1, 'n_neighbors': 5, 'weights': 'd..."


In [107]:
#best results for RandomizedSearchCV
results_random

,estimator,score,params
0,RandomForestRegressor,0.873685,"{'n_jobs': -1, 'n_estimators': 100, 'max_featu..."
1,DecisionTreeRegressor,0.747334,"{'max_features': 'auto', 'max_depth': 10, 'cri..."
2,LinearRegression,0.730396,"{'normalize': True, 'n_jobs': -1}"
3,ElasticNet,0.671531,"{'normalize': False, 'l1_ratio': 0.0}"
4,KNeighborsRegressor,0.496019,"{'weights': 'distance', 'n_neighbors': 5, 'n_j..."
